# Time Series Forecasting - Collecting Food Prices Online to Predict Inflation

Crawling Web Information Daily and Aggregating to create a time series and predict Brazilian Inflation Direction with Neural Networks (Long Short Term Memory Network)

Project had participations of Alberto Cavallo of MIT Billion Prices Project and BTG Pactual Asset Management.

Objective: 
          
     Seek alternative data online, compare daily variation, weekly, 20 days, monthly with documented monthly inflation variation of each respective food item.  
          
     Choose specific food items that historically vary and have a minimum weight of 7% on the inflation measure.
        
     These food items will be representative of a larger population or of a larger food category.
     
     4 categories (Fruits, Legumes, Greens, Fish)
    
Contacts:

     MIT Billion Prices Project - Alberto Cavallo - acavallo@mit.edu
     
     CEAGESP - Fabio Santos - fabio.santos@ceagesp.gov.br
    
App: 
     
     2 Web Crawlers daily search food prices online at two main retailers at two main states in Brazil, Sao Paulo and Belo Horizonte. 
        
     The robots generate two csv/excel files that are daily complemented automatically with the most recent date at the top of the file.
     
     Initial date of SP -  18/04/2017
     Initial date of Minas Gerais - 28/07/2017
    
Methodology

      Estimatives - obtained from the calculation of the simple arithmetic mean of prices of the locations chosen, then compared in two consecutive months, resulting in a relative mean.
      
      Agregating the relative means of the products through geometric mean, the variation of price of each subitem is obtained. This constitutes in the smallesest index aggregation that possesses explict ponderation.

      After that, Laspeyres formula is applied, obtaining all the remaining aggregation levels of inflation structure: item, subgroup, group and finally, the general index for the region. 

      The national indexes are calculated after the regional indexes, utilizing a ponderated arithmetic mean.
      
      
Flow

      Crawler Python -> Agregation & Parsing in Python -> Export DataFrame Excel -> LSTM -> Visualization PowerBI

In [ ]:
#necessary libraries

import time
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import sys
import requests
import pandas as pd
import numpy as np
import lxml.html as lh 

# Crawler Development

In [186]:
'''
Crawler 1 - Ceasa Minas
Gets daily information and exports it to csv 'CeasaMinas-daily.csv'

'''
global str
import time
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import sys
import requests
import pandas as pd
import numpy as np
import lxml.html as lh 


def doit():

    urls = 'http://www.ceasaminas.com.br/precosgeral.asp'
    
    r = requests.get(urls) # If Response 405, 400 - verify = False

    response = r.content #.text
    
    if r.status_code != 200:
        sys.exit('Non 200 status code received')
        
        
    else:
        print('Success')

                       
    soup = BeautifulSoup(response, 'lxml')

    for a in soup.find_all('table', {'class':'full_table_list'} ):
        
        df_mg = pd.DataFrame([[td.text for td in row.find_all("td")] for row in a.select("tr")])
    
    df_mg
    
    #headers = [th.text.encode("utf-8") for th in a.select("tr td")]
    #svdata = datetime.datetime.now()
    #svstr = datetime.datetime.strftime(csvdata, '%d/%m/%Y')
    
    
    with open("DF_mg.csv", "a") as f:
        wr = csv.writer(f)
        wr.writerows([[td.text for td in row.find_all("td")] for row in a.select("tr")])

#if __name__ == "__main__":
 #   while True:
    
doit()
   #     time.sleep(115200)# a cada 32h


Success


In [262]:
'''
Crawler 2 - CEAGESP
Gets daily information and gets it to csv 'ceagesp-daily.csv'

''' 

def doit():

    base = datetime.datetime.today()

    ProductinList = ['frutas', 
                     'legumes',
                     'verduras',
                     'diversos',
                     'flores',
                     'pescados',] #pegar tudo
    numdays = 2

    DatainList = [(base - datetime.timedelta(days=x)) for x in range(0, numdays)]
    
    date_strings = [dt.strftime("%d/%m/%Y") for dt in DatainList]


    for i in date_strings:
        extracted_data = []
        item = i[0:2]+i[3:5]
        for j in ProductinList:
               url = 'http://www.ceagesp.gov.br/entrepostos/servicos/cotacoes/' 
               form_data = {
                       'cot_grupo': j,
                       'cot_data':  i,
                       'submit': 'submit',
                       }

               response = requests.post(url, data=form_data)
            
               if response.status_code != 200:
                   sys.exit('Non 200 status code received')
               
        
               tree = lh.document_fromstring(response.content)

               script = tree.xpath("//tr/*/text()") 
        
               row = tree.xpath("//td[1]//*/text()[1]")
        
               mergedlist = row + script 
        
               categoria = mergedlist[1:2]
               data =  mergedlist[3:4]
            
               produto =  mergedlist[6::7]
               preco =  mergedlist[10::7]
               classif = mergedlist[7::7]
               emb = mergedlist[8::7]
            
               finalResult = dict({'Data': data, 'Categoria': categoria,'Produto': produto, 'Preco': preco, 'Class': classif, 'Embalagens': emb })
        
            #   print(finalResult)
               extracted_data.append(finalResult)
               #extracted_data.append(cols)
    
        
    stuff = pd.DataFrame(extracted_data)


    s = pd.DataFrame({'Data':np.repeat(stuff.Data.values, stuff.Produto.str.len()),
                        'Produto':np.concatenate(stuff.Produto.values)})

    a =  pd.DataFrame({'Categoria':np.repeat(stuff.Categoria.values, stuff.Preco.str.len()),
                        'Preco':np.concatenate(stuff.Preco.values)})
 
    p =  pd.DataFrame({'Class':np.repeat(stuff.Preco.values, stuff.Class.str.len()),
                            'Class':np.concatenate(stuff.Class.values)})

    y = pd.DataFrame({'Embalagens':np.repeat(stuff.Embalagens.values, stuff.Embalagens.str.len()),
                            'Embalagens':np.concatenate(stuff.Embalagens.values)})
    
    final = pd.concat([s,a,p,y], axis = 1)
     
    print(final)
    
    final['Data'] = final['Data'].str[0] #datetime
    final['Categoria'] = final['Categoria'].str[0]

    #final.to_csv("semanagetmore2807{}.csv".format(item ))
        
    with open("DF_sp.csv", "a") as f:
        final.to_csv(f, header = False)
        
#if __name__ == "__main__":
 #   while True:
doit()
#        time.sleep(115200) # a cada 32h

              Data                          Produto    Categoria  Preco  \
0    [ 02/08/2017]                  ABACATE FORTUNA    [ FRUTAS]   4,63   
1    [ 02/08/2017]                  ABACATE FORTUNA    [ FRUTAS]   4,23   
2    [ 02/08/2017]                ABACATE MARGARIDA    [ FRUTAS]   3,66   
3    [ 02/08/2017]                ABACATE MARGARIDA    [ FRUTAS]   3,33   
4    [ 02/08/2017]                  ABACATE QUINTAL    [ FRUTAS]   4,65   
5    [ 02/08/2017]                  ABACATE QUINTAL    [ FRUTAS]   4,25   
6    [ 02/08/2017]                  ABACATE AVOCADO    [ FRUTAS]   5,88   
7    [ 02/08/2017]                    ABACAXI HAVAI    [ FRUTAS]    3,7   
8    [ 02/08/2017]                    ABACAXI HAVAI    [ FRUTAS]    3,1   
9    [ 02/08/2017]                    ABACAXI HAVAI    [ FRUTAS]    2,5   
10   [ 02/08/2017]                   ABACAXI PEROLA    [ FRUTAS]   3,79   
11   [ 02/08/2017]                   ABACAXI PEROLA    [ FRUTAS]   3,29   
12   [ 02/08/2017]       

UnicodeEncodeError: 'charmap' codec can't encode character '\x80' in position 47: character maps to <undefined>

# Data Preparation and Feauture Engineering

In [264]:
#Building DataFrame Final

#parsing df_sp

df_sp = pd.read_csv('testecsv2.csv',  encoding = "ISO-8859-1")

df_sp['Preco'] = df_sp['Preco'].str.replace(',', '.')

#print(df_sp['Data'])

#parsing mg

df_mg = pd.read_csv('testemg2.csv',  encoding = "ISO-8859-1")
df_mg = df_mg.iloc[:,0:4].replace('Grande BH', '')
df_mg = pd.melt(df_mg, id_vars=["Unnamed: 0", "Produtos", "Embalagens"], 
                  var_name="Data", value_name="Preco")
    
df_mg['Data'] = df_mg['Data'].map(lambda x: x.lstrip('Grande BH'))

df_mg['Produto'] = df_mg['Produtos']

del(df_mg['Unnamed: 0'],df_mg['Produtos'] )


frames  = [df_mg, df_sp]
df_all = pd.concat(frames, keys =['Data', 'Produto', 'Embalagens'] )


df_all = df_all.rename(columns = {'Unnamed: 0':'Estado'})
df_all = df_all.iloc[:,0:7].replace('NaN', 'MG')

df_all.set_index('Data', inplace=True)

df_all.to_csv('DF_all_prices.csv')
df_all

,Categoria,Class,Embalagens,Preco,Produto,Estado
Data,,,,,,
02/08/2017,NaN,NaN,KG,3.3,ABACATE ...,MG
02/08/2017,NaN,NaN,KG,0.55,ABO. ITALIANA ...,MG
02/08/2017,NaN,NaN,KG,1.66,ABO. MENINA ...,MG
02/08/2017,NaN,NaN,KG,2,ABO. MOGANGA ...,MG
02/08/2017,NaN,NaN,DZ,33,ABACAXI ...,MG
02/08/2017,NaN,NaN,DZ,9,ALFACE ...,MG
02/08/2017,NaN,NaN,KG,14,ALHO BRASILEIRO ...,MG
02/08/2017,NaN,NaN,KG,1.25,BANANA-NANICA ...,MG
02/08/2017,NaN,NaN,KG,1.4,BANANA-PRATA ...,MG


In [255]:
#Including Weight and Inflation Variation 

from openpyxl import load_workbook
import pandas as pd

wb2 = load_workbook('C:\Crawlers Enforce\PRC_ipca.xlsx')

print(wb2.get_sheet_names())
var = wb2['Variação no mês - Brasil']
peso = wb2['Ponderação - Brasil']

df_var = pd.DataFrame(var.values)
df_peso = pd.DataFrame(peso.values)

df_peso = df_peso.rename(columns = {'1':'Produto'})
df_peso = pd.DataFrame(df_peso[1:174])

#df_peso concatenate with df_food

#check if df_food == df_var & df_var_else
df_peso

['Variação no mês - Brasil', 'Ponderação - Brasil']


,0,5,variable,value
0,NaN,NaN,2,NaN
1,NaN,NaN,2,NaN
2,NaN,NaN,2,NaN
3,NaN,NaN,2,NaN
4,NaN,NaN,2,NaN
5,NaN,NaN,2,NaN
6,NaN,NaN,2,NaN
7,NaN,NaN,2,NaN
8,NaN,NaN,2,NaN
9,NaN,NaN,2,NaN


In [13]:
#Applying LSTM Prediction
import pandas as pd

df_test = df_sp = pd.read_csv('/Users/owner/Desktop/BRA.csv')

df_test = df_test[['date','fullprice','category']]

from pandas import read_csv
from pandas import datetime
from matplotlib import pyplotdf_test[:5]

,date,fullprice,category
0,11-Oct-07,15.0,61
1,12-Oct-07,15.0,61
2,13-Oct-07,15.0,61
3,14-Oct-07,15.0,61
4,15-Oct-07,15.0,61


In [35]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
import datetime 

datetime.datetime.strptime(df_test['date'], '%d-%b-%Y').strftime('%Y-%m-%d')
#df_test['date'] = df_test['date'].astype('datetime64[ns]')
df_test['date']
df_test['fullprice'].plot()
pyplot.show()
date_list[:5]

TypeError: strptime() argument 1 must be str, not Series

In [34]:

from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
import datetime as dt

# split data into train and test

X = df_test.values
train, test = X[0:-12], X[-12:]

# walk-forward validation
history = [x for x in train]
predictions = list()
for i in range(len(test)):
# make prediction
    predictions.append(history[-1])
# observation
    history.append(test[i])
# report performance
train
#dt.datetime.strptime(test, '%d-%b-%Y').strftime('%Y-%m-%d')

rmse = sqrt(mean_squared_error(test, predictions))
#print('RMSE: %.3f' % rmse)
# line plot of observed vs predicted
#pyplot.plot(test)
#pyplot.plot(predictions)
#pyplot.show()


ValueError: could not convert string to float: '14-Oct-10'